In [1]:
obs_dim = 10
lat_dim = 4
depth = 2
groups = None
ngroups = 2
reg = None


In [2]:
import tensorflow as tf
import tf as envae


2024-10-23 03:10:14.380067: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-23 03:10:14.382712: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-23 03:10:14.391711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 03:10:14.406833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 03:10:14.411454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 03:10:14.421718: I tensorflow/core/platform/cpu_feature_gu

In [3]:
vae = envae.EnVAE(obs_dim, lat_dim, depth=depth, groups=groups, ngroups=ngroups, reg=reg)


In [4]:
inp = tf.random.uniform((6, obs_dim))
inp

<tf.Tensor: shape=(6, 10), dtype=float32, numpy=
array([[5.18002391e-01, 5.56159377e-01, 5.89952230e-01, 6.81097627e-01,
        1.83026433e-01, 7.41274595e-01, 1.10490799e-01, 4.57819939e-01,
        1.79038405e-01, 2.01516986e-01],
       [5.65463305e-01, 9.61239696e-01, 7.81677842e-01, 6.30607128e-01,
        9.07888055e-01, 6.27601624e-01, 8.76237035e-01, 4.88314986e-01,
        5.22412062e-02, 7.58236885e-01],
       [8.44390512e-01, 2.98495531e-01, 8.59706283e-01, 1.32871151e-01,
        2.62367725e-01, 4.04365063e-01, 4.10044193e-03, 4.22021389e-01,
        2.51849890e-01, 7.28822947e-01],
       [2.59844303e-01, 6.47765398e-02, 5.28742790e-01, 7.68471599e-01,
        1.24877095e-01, 5.33072710e-01, 2.39372253e-04, 6.27292275e-01,
        1.44092441e-01, 2.10564733e-01],
       [3.27432036e-01, 2.09649801e-02, 4.59413052e-01, 3.37758064e-02,
        4.57769632e-01, 1.17800117e-01, 8.07035565e-01, 9.72942114e-02,
        5.94029784e-01, 1.95814252e-01],
       [7.89202452e-01, 7.

In [5]:
tf.stack([
        tf.repeat(tf.range(inp.shape[0])[..., tf.newaxis], vae._groups[0].shape[0], axis=1),
        tf.repeat(vae._groups[0][tf.newaxis, ...], inp.shape[0], axis=0)
    ], axis=2)

<tf.Tensor: shape=(6, 5, 2), dtype=int32, numpy=
array([[[0, 0],
        [0, 2],
        [0, 4],
        [0, 6],
        [0, 8]],

       [[1, 0],
        [1, 2],
        [1, 4],
        [1, 6],
        [1, 8]],

       [[2, 0],
        [2, 2],
        [2, 4],
        [2, 6],
        [2, 8]],

       [[3, 0],
        [3, 2],
        [3, 4],
        [3, 6],
        [3, 8]],

       [[4, 0],
        [4, 2],
        [4, 4],
        [4, 6],
        [4, 8]],

       [[5, 0],
        [5, 2],
        [5, 4],
        [5, 6],
        [5, 8]]], dtype=int32)>

In [6]:
tf.gather_nd(
    inp,
    tf.stack([
        # tf.repeat(tf.range(inp.shape[0])[..., tf.newaxis], vae._groups[0].shape[0], axis=1),
        tf.broadcast_to(tf.range(inp.shape[0])[..., tf.newaxis], (inp.shape[0], vae._groups[0].shape[0])),
        tf.repeat(vae._groups[0][tf.newaxis, ...], inp.shape[0], axis=0)
    ], axis=2),
)

<tf.Tensor: shape=(6, 5), dtype=float32, numpy=
array([[5.18002391e-01, 5.89952230e-01, 1.83026433e-01, 1.10490799e-01,
        1.79038405e-01],
       [5.65463305e-01, 7.81677842e-01, 9.07888055e-01, 8.76237035e-01,
        5.22412062e-02],
       [8.44390512e-01, 8.59706283e-01, 2.62367725e-01, 4.10044193e-03,
        2.51849890e-01],
       [2.59844303e-01, 5.28742790e-01, 1.24877095e-01, 2.39372253e-04,
        1.44092441e-01],
       [3.27432036e-01, 4.59413052e-01, 4.57769632e-01, 8.07035565e-01,
        5.94029784e-01],
       [7.89202452e-01, 4.05499220e-01, 9.24327254e-01, 5.67498446e-01,
        7.46827722e-01]], dtype=float32)>

In [7]:
enc = vae.encode(inp)
enc

(<tf.Tensor: shape=(6, 4, 2), dtype=float32, numpy=
 array([[[-0.18475778,  0.11742005],
         [ 0.3510018 ,  0.24868369],
         [ 0.4104907 ,  0.33070672],
         [-0.05704182, -0.04232877]],
 
        [[-0.73895097, -0.2530008 ],
         [ 0.6958515 , -0.39121246],
         [ 0.6746117 , -0.1966413 ],
         [ 0.00875787,  0.54525656]],
 
        [[-0.14277005,  0.20851043],
         [ 0.4841404 ,  0.41507286],
         [ 0.6092447 ,  0.55698854],
         [-0.0607367 , -0.08729643]],
 
        [[-0.07344975,  0.08357716],
         [ 0.18139414,  0.18659696],
         [ 0.17369232,  0.26778802],
         [-0.01069911, -0.00406505]],
 
        [[-0.2817197 , -0.26470602],
         [ 0.42656535, -0.26176387],
         [ 0.47887206, -0.2727465 ],
         [ 0.04881392,  0.3944119 ]],
 
        [[-0.2485918 , -0.2334058 ],
         [ 0.29194582, -0.32006067],
         [ 0.36731303, -0.24773514],
         [ 0.09356235,  0.41889793]]], dtype=float32)>,
 <tf.Tensor: shape=(6, 4, 

In [8]:
# experts = [0, 1]


# req_means = tf.gather_nd(
#     means,
#     tf.constant(
#         [
#             [[[b, h, g] for g in experts] for h in range(means.shape[1])]
#             for b in range(means.shape[0])
#         ],
#         dtype=tf.int32,
#     ),
# )


In [9]:
samp = vae.reparameterize(*enc)
samp

<tf.Tensor: shape=(6, 4), dtype=float32, numpy=
array([[ 0.15750298, -0.23241344, -0.5506697 , -0.34435776],
       [-0.87570065, -0.27371392, -0.7968709 , -0.56052434],
       [-0.00380526,  1.0624281 ,  0.31215838, -0.28882527],
       [-1.0352316 , -0.08443291, -0.3910225 ,  0.974382  ],
       [-1.1221995 ,  1.0806416 ,  0.20950891,  1.2247734 ],
       [-0.5357095 , -0.1942316 , -0.6696874 ,  1.0954849 ]],
      dtype=float32)>

In [10]:
inp

<tf.Tensor: shape=(6, 10), dtype=float32, numpy=
array([[5.18002391e-01, 5.56159377e-01, 5.89952230e-01, 6.81097627e-01,
        1.83026433e-01, 7.41274595e-01, 1.10490799e-01, 4.57819939e-01,
        1.79038405e-01, 2.01516986e-01],
       [5.65463305e-01, 9.61239696e-01, 7.81677842e-01, 6.30607128e-01,
        9.07888055e-01, 6.27601624e-01, 8.76237035e-01, 4.88314986e-01,
        5.22412062e-02, 7.58236885e-01],
       [8.44390512e-01, 2.98495531e-01, 8.59706283e-01, 1.32871151e-01,
        2.62367725e-01, 4.04365063e-01, 4.10044193e-03, 4.22021389e-01,
        2.51849890e-01, 7.28822947e-01],
       [2.59844303e-01, 6.47765398e-02, 5.28742790e-01, 7.68471599e-01,
        1.24877095e-01, 5.33072710e-01, 2.39372253e-04, 6.27292275e-01,
        1.44092441e-01, 2.10564733e-01],
       [3.27432036e-01, 2.09649801e-02, 4.59413052e-01, 3.37758064e-02,
        4.57769632e-01, 1.17800117e-01, 8.07035565e-01, 9.72942114e-02,
        5.94029784e-01, 1.95814252e-01],
       [7.89202452e-01, 7.

In [11]:
tf.gather_nd(inp, [[0], [2], [3]])

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[5.18002391e-01, 5.56159377e-01, 5.89952230e-01, 6.81097627e-01,
        1.83026433e-01, 7.41274595e-01, 1.10490799e-01, 4.57819939e-01,
        1.79038405e-01, 2.01516986e-01],
       [8.44390512e-01, 2.98495531e-01, 8.59706283e-01, 1.32871151e-01,
        2.62367725e-01, 4.04365063e-01, 4.10044193e-03, 4.22021389e-01,
        2.51849890e-01, 7.28822947e-01],
       [2.59844303e-01, 6.47765398e-02, 5.28742790e-01, 7.68471599e-01,
        1.24877095e-01, 5.33072710e-01, 2.39372253e-04, 6.27292275e-01,
        1.44092441e-01, 2.10564733e-01]], dtype=float32)>